In [ ]:
import openpyxl
import polars as pl

print("load")
wb = openpyxl.load_workbook('data.xlsx')
print("loaded")
ws = wb.active

label = [str(ws.cell(row=i,column=1).value) for i in range(1+1,ws.max_row+1)]
titre = [ws.cell(row=i,column=2).value for i in range(1+1,ws.max_row+1)]
texte = [ws.cell(row=i,column=3).value for i in range(1+1,ws.max_row+1)]

ids = list(range(0,len(label)))


print(label)
print(titre)
print(texte)
df = pl.DataFrame(
    {   "label" : label,
        "titre" : titre,
        "texte": texte,
        "id" : ids
    }
)

print(df)

print(df[0,1])

print(len(df))

id2label = dict(zip(ids, label))
label2id = dict(zip(label, ids))

print(id2label)
print(label2id)

In [32]:
import ollama

df_questions = pl.DataFrame(
    {   "id" : [],
        "question" : [],
    },
    schema=[("id", pl.Int64), ("question", pl.String)]
    )

n = len(df)
model='wizardlm2:7b'

for i in range(0, n):

    prompt = """Tu aides à la création d'un jeu d'enquête nommé "Dieu est mort".
    Le principe du jeu est le suivant : Dieu est mort, quatre archanges et cinq princes-démons figurent sur la liste des suspects.
    Chaque joueur incarne un archange ou un prince-démon. Les joueurs doivent découvrir ce qui est arrivé à Dieu, en discutant entre eux et en obtenant des indices. 
    Sans mentionner l'indice, crée 20 possibles questions à poser au maître du jeu pour obtenir l'indice suivant : """ + str(df[i,1]) + " : " + str(df[i,2])

    generated = ollama.generate(model=model, #choisir un modèle
                            prompt=prompt)

    id = str(df[i,3])
    res = generated["response"]
    res_list = res.split("\n")

    id_arr = [int(id)] * len(res_list)
    print(id_arr)

    df_temp = pl.DataFrame(
    {   "id" : id_arr,
        "question" : res_list,
    }
    )

    df_questions = pl.concat(
        [df_questions, df_temp]
    )

print(df_questions)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10

In [33]:
import polars as pl

path = "questions_" + model + ".csv"
                    
df_questions.write_csv(path,
                       separator=";")

In [ ]:
# POUR GENERER DES QUESTIONS SUPPLEMENTAIRES POUR UN INDICE SPECIFIQUE

import ollama

df_questions = pl.DataFrame(
    {   "id" : [],
        "question" : [],
    },
    schema=[("id", pl.Int64), ("question", pl.String)]
    )

i = 5
model='wizardlm2:7b' # choisir modèle

prompt = """Tu aides à la création d'un jeu d'enquête nommé "Dieu est mort".
Le principe du jeu est le suivant : Dieu est mort, quatre archanges et cinq princes-démons figurent sur la liste des suspects.
Chaque joueur incarne un archange ou un prince-démon. Les joueurs doivent découvrir ce qui est arrivé à Dieu, en discutant entre eux et en obtenant des indices. 
Sans mentionner l'indice, crée 5 possibles questions à poser au maître du jeu pour obtenir l'indice suivant : """ + str(df[i,1]) + " : " + str(df[i,2])

print(prompt)

generated = ollama.generate(model=model,
                        prompt=prompt)

id = str(df[i,3])
res = generated["response"]
res_list = res.split("\n")

id_arr = [int(id)] * len(res_list)
print(id_arr)

df_temp = pl.DataFrame(
{   "id" : id_arr,
    "question" : res_list,
}
)

df_questions = pl.concat(
    [df_questions, df_temp]
)

print(df_questions)

path = "questions_" + str(i) + "_" + model + ".csv"
                    
df_questions.write_csv(path,
                       separator=";")

NOTES

Modèles utilisés pour la génération de questions :
- Openhermes 2.5 7B
- Gemma 7B
- Gemma 2 9B
- Gemma 2 27B
- Mistral-Nemo
- Orca2 13B
- WizardLM2 7B


Nettoyage : 
- Assembler les multiples csv en un seul nommé "questions_all.csv". Nettoyage manuel possiblement nécessaire (supprimer numéros de questions, signes et lignes inutiles, espaces en trop...)

In [34]:
import polars as pl 
df_questions = pl.read_csv(source="questions_all.csv", 
                           has_header=True,
                           separator= ';')
df_questions.columns = ["label", "text"]

print(df_questions)

num_labels = len(df_questions["label"].unique())
print(num_labels)

print(df_questions["label"].unique().sort())


shape: (3_164, 2)
┌───────┬─────────────────────────────────┐
│ label ┆ text                            │
│ ---   ┆ ---                             │
│ i64   ┆ str                             │
╞═══════╪═════════════════════════════════╡
│ 0     ┆ Quelle était la nature exacte … │
│ 0     ┆ Quels facteurs ont été pris en… │
│ 0     ┆ Les résultats ont-ils révélé u… │
│ 0     ┆ La puissance des créations de … │
│ 0     ┆ Quelles méthodes spécifiques o… │
│ …     ┆ …                               │
│ 21    ┆ Quelles sont les lois ou tabou… │
│ 21    ┆ Comment la grossesse de Marion… │
│ 21    ┆ Quelles sont les mesures prise… │
│ 21    ┆ Comment les autres suspects pe… │
│ 21    ┆ De quelle manière l'énigme de … │
└───────┴─────────────────────────────────┘
22
shape: (22,)
Series: 'label' [i64]
[
	0
	1
	2
	3
	4
	…
	17
	18
	19
	20
	21
]


In [35]:
from datasets import Dataset
from sklearn.model_selection import train_test_split


train, test = train_test_split(df_questions, test_size=0.1)

train, eval = train_test_split(train, test_size=0.1)

eval_db = eval["text"]

x = []
for i in range(len(eval_db)):
    y = eval_db[i][:512]
    x.append(y)
eval_db = x

data_train = Dataset.from_polars(train,
                           split="train")

data_test = Dataset.from_polars(test,
                           split="test")

print(data_train)
print(data_train["label"])
print(data_train["text"])

Dataset({
    features: ['label', 'text'],
    num_rows: 2562
})
[12, 20, 20, 17, 2, 13, 13, 10, 17, 18, 10, 9, 6, 6, 12, 11, 12, 1, 18, 9, 3, 0, 20, 4, 0, 20, 16, 11, 13, 20, 1, 5, 19, 10, 17, 9, 21, 6, 11, 19, 18, 4, 8, 18, 5, 21, 20, 21, 2, 8, 17, 8, 17, 4, 16, 6, 9, 1, 7, 9, 8, 16, 17, 0, 8, 13, 5, 6, 4, 6, 2, 3, 16, 17, 16, 19, 18, 3, 18, 20, 4, 2, 21, 16, 1, 15, 20, 3, 16, 17, 10, 7, 0, 21, 16, 13, 20, 16, 2, 11, 5, 0, 15, 20, 16, 9, 18, 17, 2, 17, 0, 4, 10, 3, 13, 1, 20, 10, 5, 9, 21, 2, 17, 10, 6, 21, 8, 17, 20, 2, 0, 6, 20, 13, 0, 13, 15, 6, 12, 12, 11, 11, 19, 13, 5, 18, 2, 0, 0, 21, 13, 21, 14, 21, 13, 6, 7, 7, 14, 5, 8, 21, 21, 5, 20, 20, 9, 15, 10, 20, 8, 15, 16, 8, 14, 5, 10, 2, 16, 19, 6, 10, 21, 19, 6, 5, 6, 19, 18, 6, 2, 15, 3, 1, 13, 20, 7, 10, 21, 19, 3, 18, 4, 13, 0, 11, 17, 10, 16, 13, 16, 8, 9, 21, 10, 16, 4, 5, 7, 16, 7, 7, 2, 1, 8, 10, 6, 14, 16, 4, 0, 17, 13, 7, 9, 8, 8, 16, 19, 4, 14, 16, 4, 9, 6, 0, 6, 20, 21, 20, 4, 11, 17, 10, 7, 14, 0, 15, 14, 21, 12, 8, 3

In [36]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./distilbert-base-multilingual-cased")

def preprocess_function(database):
    return tokenizer(database["text"], 
                     truncation=True
                     )


tokenized_train = data_train.map(preprocess_function, batched=True)
tokenized_test = data_test.map(preprocess_function, batched=True)

print(tokenized_train)

print(tokenized_train["text"])
print(tokenized_train["label"])
print(tokenized_train["input_ids"])
print(tokenized_train["attention_mask"])



/home/ds/classification_jeu/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████| 317/317 [00:00<00:00, 24739.86 examples/s]

Dataset({
    features: ['label', 'text', 'input_ids', 'attention_mask'],
    num_rows: 2562
})
["Qui était le Prince Démon Valefor et pourquoi l'Archange Janus passait-il du temps dans ses appartements ? Y a-t-il un lien entre lui et Janus ?", 'Pourquoi Malphas a-t-il décidé de traiter cet Antéchrist différemment des autres nouveaux arrivants ?', "Q: Quand est-ce que l'Antéchrist est arrivé en Enfer?", "Comment les suspects ont-ils réagi à l'annonce de la mort de Dieu ?", "Avez-vous d'autres informations sur Maria que vous pourriez partager avec nous ?", "Y a-t-il des indices sur les raisons pour lesquelles l'individu à la pique a disparu après sa première attaque ?", 'Y a-t-il eu des tensions ou des rivalités entre certains archanges et princes démons avant cet événement ?', 'Existe-t-il un rapport entre les problèmes administratifs en Enfer et le décès de Dieu ?', "Comment les autres archange se sont-ils répartis à l'époque de l'enquête Pélican, et quelle était la position de Janus 

In [37]:
id2titre = dict(zip(ids, titre))
titre2id = dict(zip(titre, ids))
print(id2titre)
print(titre2id)

print(id2label)
print(label2id)

{0: "Etudes scientifiques menées par l'archange Yves", 1: 'Apparitions étranges à the Scientific and Magic Institute, San Franscico, USA', 2: 'Ragots d’une petite fille', 3: 'Alerte sur le continuum spatiotemporel', 4: "Le nouveau virus de l'Archange Guy", 5: "L'intervention infernale en Yougoslavie", 6: 'Un système de protection des Archanges', 7: 'Une rencontre entre Archanges et Princes Démons sur Terre', 8: "Un vol chez l'archange Yves", 9: 'Destruction du Vatican', 10: 'Problèmes administratifs en Enfer', 11: "Ragots d'un petit démon", 12: 'Emploi du temps des archanges dossier C42', 13: 'Rapport de police, incident du bar dit « chez petit Louis »', 14: "Rapport du démon Jukseksakj sur l'activité éthérique anormale de l'hôtel Ragazza à Milan", 15: 'L’incident de l’escalier', 16: 'Des remous en enfer : conscription et entrainement', 17: 'Le dossier Pélican', 18: 'Un nouveau texte évangélique', 19: 'Les passes temps de Lucifer', 20: 'Un nouvel arrivant', 21: 'Une heureuse nouvelle'}

In [38]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

model = AutoModelForSequenceClassification.from_pretrained(
    "./distilbert-base-multilingual-cased",
    num_labels = num_labels,
    id2label=id2titre, 
    label2id=titre2id)

training_args = TrainingArguments(

    output_dir="model_classif",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=16,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False

)

trainer = Trainer(

    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer
)

trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at ./distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  6%|▋         | 161/2576 [00:09<02:25, 16.62it/s]

{'eval_loss': 2.1829607486724854, 'eval_runtime': 0.1694, 'eval_samples_per_second': 1871.598, 'eval_steps_per_second': 118.082, 'epoch': 1.0}


 12%|█▎        | 322/2576 [00:20<02:21, 15.98it/s]

{'eval_loss': 1.3730756044387817, 'eval_runtime': 0.1316, 'eval_samples_per_second': 2409.707, 'eval_steps_per_second': 152.032, 'epoch': 2.0}


 19%|█▉        | 483/2576 [00:30<01:32, 22.66it/s]

...

100%|██████████| 2576/2576 [02:49<00:00, 21.68it/s]

{'eval_loss': 0.7627058029174805, 'eval_runtime': 0.1393, 'eval_samples_per_second': 2275.436, 'eval_steps_per_second': 143.561, 'epoch': 16.0}


100%|██████████| 2576/2576 [02:52<00:00, 14.97it/s]

{'train_runtime': 172.068, 'train_samples_per_second': 238.231, 'train_steps_per_second': 14.971, 'train_loss': 0.5151921929965108, 'epoch': 16.0}


TrainOutput(global_step=2576, training_loss=0.5151921929965108, metrics={'train_runtime': 172.068, 'train_samples_per_second': 238.231, 'train_steps_per_second': 14.971, 'total_flos': 522045071427384.0, 'train_loss': 0.5151921929965108, 'epoch': 16.0})

In [41]:
from transformers import pipeline
classifier = pipeline("text-classification", 
                      model = "./model_classif/checkpoint-2576/", 
                      device = "cuda")

result = classifier("Que s'est-il passé en yougoslavie ?")
print(result)

[{'label': "L'intervention infernale en Yougoslavie", 'score': 0.9773578643798828}]


In [44]:
from transformers import pipeline
import polars as pl
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

classifier = pipeline("text-classification", 
                      model = "./model_classif/checkpoint-2576/", 
                      device = "cuda")

result = classifier(eval_db)
print(result)
pred = []

for i in range(len(result)):
    x = result[i]["label"]
    pred.append(x)

eval = pl.DataFrame(eval)
eval = eval.select("label").cast(pl.String)
true = eval.with_columns(label = pl.col("label").replace(id2titre))

df_results = pl.DataFrame({
    "true":true,
    "pred":pred

})

conf = confusion_matrix(true, pred)

accuracy = accuracy_score(true, pred)

print(accuracy)

report = classification_report(true, pred)

print(report)

[{'label': 'Des remous en enfer : conscription et entrainement', 'score': 0.9903286099433899}, {'label': "Le nouveau virus de l'Archange Guy", 'score': 0.9959551095962524}, {'label': "L'intervention infernale en Yougoslavie", 'score': 0.9912410974502563}, {'label': 'Un système de protection des Archanges', 'score': 0.9957600235939026}, {'label': 'Un nouveau texte évangélique', 'score': 0.9972000122070312}, {'label': 'L’incident de l’escalier', 'score': 0.9927946925163269}, {'label': "Un vol chez l'archange Yves", 'score': 0.9781306385993958}, {'label': "Un vol chez l'archange Yves", 'score': 0.9938005805015564}, {'label': 'Une heureuse nouvelle', 'score': 0.9095145463943481}, {'label': 'Emploi du temps des archanges dossier C42', 'score': 0.9954276084899902}, {'label': 'Un nouveau texte évangélique', 'score': 0.9970147609710693}, {'label': 'Des remous en enfer : conscription et entrainement', 'score': 0.9897059202194214}, {'label': 'Les passes temps de Lucifer', 'score': 0.990910112857